In [8]:
!pip install astpretty

In [9]:
!ls data

evaluation training


In [10]:
import numpy as np
from functools import reduce
from collections import namedtuple
from typing import List, Tuple, Union,Dict
import json
import re

MAXSIDE = 100
MAXAREA = 40 * 40
MAXPIXELS = 40 * 40 * 5

import sys
import os

sys.path.append(os.path.abspath("/Users/saisur/Desktop/projects/arc/arcsol/arc-dsl"))


from dsl import *
from solvers import *

In [11]:
# sys.path.append(os.path.abspath(os.path.join('/Users/saisur/Desktop/projects/arc/arclang/src/arclang')))
# sys.path.append(os.path.abspath(os.path.join('/Users/saisur/Desktop/projects/arc/arclang/src/')))

# from image import Image, Point
# from function import *

In [12]:
path = "data/training/e6721834.json"
with open(path,"r") as f:
    data_ce9 = json.loads(f.read())
img_mask = data_ce9["train"][0]["input"]
# img_mask = np.array(img_mask)
# m,n = img_mask.shape
# input_img= Image(0,0,n,m,img_mask)
# display_matrix(input_img)

In [13]:
img = tuple([tuple(i) for i in img_mask])

In [26]:
import ast
from typing import Any, List

class Node:
    def __init__(self, node_type: str, value: Any = None, children: List['Node'] = None):
        self.node_type = node_type
        self.value = value
        self.children = children or []

    def __repr__(self):
        return f"Node(type={self.node_type}, value={self.value}, children={self.children})"

    def format(self, indent=0):
        indent_str = '    ' * indent
        result = f"{indent_str}Node(type={self.node_type}"
        
        if self.value is not None:
            result += f", value={self.value}"
        
        if self.children:
            result += ", children=[\n"
            for child in self.children:
                result += child.format(indent + 1) + ",\n"
            result += f"{indent_str}]"
        else:
            result += ", children=[]"
        
        result += ")"
        return result

class CustomASTVisitor(ast.NodeVisitor):
    def generic_visit(self, node):
        node_type = type(node).__name__
        children = []
        value = None

        for field, value in ast.iter_fields(node):
            if isinstance(value, ast.AST):
                children.append(self.visit(value))
            elif isinstance(value, list):
                for item in value:
                    if isinstance(item, ast.AST):
                        children.append(self.visit(item))

        if hasattr(node, 'id'):
            value = node.id
        elif hasattr(node, 'name'):
            value = node.name
        elif hasattr(node, 'arg'):
            value = node.arg
        elif isinstance(node, ast.Str):
            value = node.s
        elif isinstance(node, ast.Num):
            value = node.n

        return Node(node_type, value, children)

def generate_custom_ast(code: str) -> Node:
    tree = ast.parse(code)
    visitor = CustomASTVisitor()
    return visitor.visit(tree)



In [33]:
import ast
from typing import Any, List, Union

class Node:
    def __init__(self, node_type: str, value: Any = None, children: List['Node'] = None):
        self.node_type = node_type
        self.value = value
        self.children = children or []

    def __repr__(self):
        return f"Node(type={self.node_type}, value={self.value}, children={self.children})"

    def format(self, indent=0):
        indent_str = '    ' * indent
        result = f"{indent_str}Node(type={self.node_type}"
        
        if self.value is not None:
            result += f", value={self.value}"
        
        if self.children:
            result += ", children=[\n"
            for child in self.children:
                result += child.format(indent + 1) + ",\n"
            result += f"{indent_str}]"
        else:
            result += ", children=[]"
        
        result += ")"
        return result

class CustomASTVisitor(ast.NodeVisitor):
    def visit_FunctionDef(self, node):
        params = [arg.arg for arg in node.args.args]
        return_type = self.visit(node.returns) if node.returns else None
        docstring = ast.get_docstring(node)
        body = [self.visit(n) for n in node.body if not isinstance(n, ast.Expr) or not isinstance(n.value, ast.Str)]
        return Node("FUNCTION_DEF", value={
            'name': node.name,
            'params': params,
            'return_type': return_type,
            'docstring': docstring
        }, children=body)

    def visit_Assign(self, node):
        target = node.targets[0].id if isinstance(node.targets[0], ast.Name) else self.visit(node.targets[0])
        value = self.visit(node.value)
        return Node("ASSIGN", value={'variable': target, 'expression': value})

    def visit_Call(self, node):
        func_name = node.func.id if isinstance(node.func, ast.Name) else self.visit(node.func)
        args = [self.visit(arg) for arg in node.args]
        return Node("FUNCTION", value={'name': func_name, 'args': args})

    def visit_Name(self, node):
        return node.id

    def visit_Num(self, node):
        return node.n

    def visit_Str(self, node):
        return node.s

    def visit_List(self, node):
        return [self.visit(elt) for elt in node.elts]

    def visit_Tuple(self, node):
        return tuple(self.visit(elt) for elt in node.elts)

    def visit_Set(self, node):
        return {self.visit(elt) for elt in node.elts}

    def visit_Dict(self, node):
        return {self.visit(k): self.visit(v) for k, v in zip(node.keys, node.values)}

    def visit_BinOp(self, node):
        left = self.visit(node.left)
        right = self.visit(node.right)
        op = type(node.op).__name__
        return f"{left} {op} {right}"

    def visit_Compare(self, node):
        left = self.visit(node.left)
        ops = [type(op).__name__ for op in node.ops]
        comparators = [self.visit(c) for c in node.comparators]
        return f"{left} {' '.join(f'{op} {comp}' for op, comp in zip(ops, comparators))}"

    def visit_For(self, node):
        target = self.visit(node.target)
        iter = self.visit(node.iter)
        body = [self.visit(n) for n in node.body]
        return Node("FOR", value={'target': target, 'iter': iter}, children=body)

    def visit_If(self, node):
        test = self.visit(node.test)
        body = [self.visit(n) for n in node.body]
        orelse = [self.visit(n) for n in node.orelse] if node.orelse else None
        return Node("IF", value={'test': test}, children=body + (orelse or []))

    def visit_Return(self, node):
        value = self.visit(node.value) if node.value else None
        return Node("RETURN", value=value)

    def generic_visit(self, node):
        return type(node).__name__

def parse_code(code: str) -> Node:
    tree = ast.parse(code)
    visitor = CustomASTVisitor()
    return Node("ROOT", children=[visitor.visit(node) for node in tree.body])

# Example usage
txt = '''def vperiod(
    obj: Object
) -> Integer:
    """ vertical periodicity """
    normalized = normalize(obj)
    h = height(normalized)
    for p in range(1, h):
        offsetted = shift(normalized, (-p, 0))
        pruned = frozenset({(c, (i, j)) for c, (i, j) in offsetted if i >= 0})
        if pruned.issubset(normalized):
            return p
    return h'''

ast = parse_code(txt)
print(ast.format())

Node(type=ROOT, children=[
    Node(type=FUNCTION_DEF, value={'name': 'vperiod', 'params': ['obj'], 'return_type': 'Integer', 'docstring': 'vertical periodicity '}, children=[
        Node(type=ASSIGN, value={'variable': 'normalized', 'expression': Node(type=FUNCTION, value={'name': 'normalize', 'args': ['obj']}, children=[])}, children=[]),
        Node(type=ASSIGN, value={'variable': 'h', 'expression': Node(type=FUNCTION, value={'name': 'height', 'args': ['normalized']}, children=[])}, children=[]),
        Node(type=FOR, value={'target': 'p', 'iter': Node(type=FUNCTION, value={'name': 'range', 'args': [1, 'h']}, children=[])}, children=[
            Node(type=ASSIGN, value={'variable': 'offsetted', 'expression': Node(type=FUNCTION, value={'name': 'shift', 'args': ['normalized', ('UnaryOp', 0)]}, children=[])}, children=[]),
            Node(type=ASSIGN, value={'variable': 'pruned', 'expression': Node(type=FUNCTION, value={'name': 'frozenset', 'args': ['SetComp']}, children=[])}, chi

In [30]:
txt

'def vperiod(\n    obj: Object\n) -> Integer:\n    """ vertical periodicity """\n    normalized = normalize(obj)\n    h = height(normalized)\n    for p in range(1, h):\n        offsetted = shift(normalized, (-p, 0))\n        pruned = frozenset({(c, (i, j)) for c, (i, j) in offsetted if i >= 0})\n        if pruned.issubset(normalized):\n            return p\n    return h'

In [32]:
custom_ast = parse_code(txt)

# Print the formatted custom AST
print(custom_ast.format())

AttributeError: 'NoneType' object has no attribute 'children'

In [14]:
solve_e6721834(img)

((0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0),
 (0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 2, 1, 1, 1, 0, 0),
 (0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 1, 1, 1, 2, 1, 0, 0),
 (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0),
 (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0),
 (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0),
 (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0),
 (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0),
 (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0),
 (0, 0, 1, 1, 3, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0),
 (0, 0, 1, 3, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0),
 (0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0),
 (0, 0, 1, 3, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0),
 (0, 0, 1, 1, 3, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0),
 (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0))

In [15]:
import ast
import astpretty

def generate_ast(code):
    tree = ast.parse(code)
    return tree

def print_ast(tree):
    astpretty.pprint(tree)

In [31]:
txt = '''def vperiod(
    obj: Object
) -> Integer:
    """ vertical periodicity """
    normalized = normalize(obj)
    h = height(normalized)
    for p in range(1, h):
        offsetted = shift(normalized, (-p, 0))
        pruned = frozenset({(c, (i, j)) for c, (i, j) in offsetted if i >= 0})
        if pruned.issubset(normalized):
            return p
    return h'''

In [23]:
print_ast(a)

Module(
    body=[
        FunctionDef(
            lineno=1,
            col_offset=0,
            end_lineno=12,
            end_col_offset=12,
            name='vperiod',
            args=arguments(
                posonlyargs=[],
                args=[
                    arg(
                        lineno=2,
                        col_offset=4,
                        end_lineno=2,
                        end_col_offset=15,
                        arg='obj',
                        annotation=Name(lineno=2, col_offset=9, end_lineno=2, end_col_offset=15, id='Object', ctx=Load()),
                        type_comment=None,
                    ),
                ],
                vararg=None,
                kwonlyargs=[],
                kw_defaults=[],
                kwarg=None,
                defaults=[],
            ),
            body=[
                Expr(
                    lineno=4,
                    col_offset=4,
                    end_lineno=4,
                   